# Matcher

In [ ]:
#| default_exp matcher
%load_ext autoreload
%autoreload 2

### Overview
The Parser outputs a DiGraph, defined by a pattern string (LHS). Nodes in the pattern graph have the same name as in the pattern string.

The **Matcher** receives an input graph, along with the Parser-generated pattern graph. It finds LHS matches in the searched graph, and returns a list of dictionaries - each representing a single match - whose keys are the names of the pattern-defined nodes, and values are the names of the actual matching nodes in the searched graph.

The list of mappings is returned by the Matcher's main function, **find_matches**.

### Requirements

In [ ]:
#| export
from networkx import DiGraph
from networkx.algorithms import isomorphism # check subgraph's isom.
import itertools # iterating over all nodes\edges combinations
from typing import *

### Find Matches
Given an input graph and a pattern graph, returns a list of mappings from pattern nodes to actual nodes, each representing a single match.

In [ ]:
#| export
def find_matches(input: DiGraph, pattern: DiGraph) -> List[Dict[str, Hashable]]:
    """
    Finds matches of a pattern graph inside an input graph, and returns mappings for each match.

    Parameters
    ----------
    input:
        A DiGraph to be scanned for finding pattern matches.
    pattern:
        A DiGraph representing a graph pattern, in the same format as the output of Parser module.

    Returns
    -------
    A list of str->hashable dictionaries:
        List of mappings from pattern nodes names to actual nodes names.
    """
    def attributes_exist(original: dict, pattern: dict):
        for attr_name, _ in pattern.items():
            if attr_name not in original:
                return False
        return True

    # Narrow down search space by removing irrelavant nodes
    matching_nodes = set()
    for (_, pattern_attr) in pattern.nodes(data=True):
        for (graph_node, graph_attr) in input.nodes(data=True):
            if attributes_exist(graph_attr, pattern_attr):
                matching_nodes.add(graph_node)
    reduced_input: DiGraph = input.subgraph(matching_nodes)

    isom_matches = []
    for sub_nodes in itertools.combinations(reduced_input.nodes, len(pattern.nodes)):
        nodes_subg: DiGraph = reduced_input.subgraph(sub_nodes) # only the selected nodes and connected edges
        for sub_edges in itertools.combinations(nodes_subg.edges(data=True), len(pattern.edges)): # subset of the connected edges
            # Create a subgraph with selected edges and nodes
            subg = DiGraph()
            subg.add_nodes_from(list(nodes_subg.nodes(data=True)))
            subg.add_edges_from(list(sub_edges))

            # Find structural matches with the selected edges and nodes
            matcher = isomorphism.DiGraphMatcher(pattern, subg)
            for isom_mapping in matcher.isomorphisms_iter():
                isom_matches.append((nodes_subg, isom_mapping))

    # Find true matches among isoms (matche pattern's attributes)
    true_matches = []
    for (subgraph, mapping) in isom_matches:
        is_match = True
        # check nodes match
        for (pattern_node, original_node) in mapping.items():
            if not attributes_exist(subgraph.nodes[original_node], pattern.nodes[pattern_node]):
                is_match = False
                break
        if not is_match:
            continue

        # check edges match
        for edge in pattern.edges(data=True):
            pattern_attr = edge[2]
            original_attr = subgraph.edges[mapping[edge[0]], mapping[edge[1]]]
            if not attributes_exist(original_attr, pattern_attr):
                is_match = False
                break
        if is_match:
            true_matches.append(mapping)

    return true_matches

### Tests

#### Test Utils

In [ ]:
def create_graph(nodes, edges):
    g = DiGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    return g

def test_match(input, pattern, expected):
    res = find_matches(input, pattern)
    for x in res:
        if x not in expected:
            return False  
    for x in expected:
        if x not in res:
            return False
    return True

#### Test Cases

In [ ]:
# Simple graph tests
input = create_graph(
    ['A','B','C','D'], 
    [
        ('A','B'),
        ('A','C'),
        ('A', 'A'),
        ('C', 'C'),
        ('A', 'C')
    ]
)

pattern1 = create_graph(['X'], [('X', 'X')])
assert test_match(input, pattern1, [{'X': 'A'}, {'X': 'C'}])
assert test_match(input, pattern1, [{'X': 'C'}, {'X': 'A'}])
assert not test_match(input, pattern1, [{'X': 'A'}])

pattern2 = create_graph(['X', 'Y'], [('X', 'X')])
assert test_match(input, pattern2, [{'X': 'A', 'Y': 'B'}, {'X': 'A', 'Y': 'C'}, {'X': 'A', 'Y': 'D'}, 
                                    {'X': 'C', 'Y': 'A'}, {'X': 'C', 'Y': 'B'}, {'X': 'C', 'Y': 'D'}])

pattern3 = create_graph(['X', 'Y'], [('X', 'X'), ('X', 'Y')])
assert test_match(input, pattern3, [{'X': 'A', 'Y': 'B'}, {'X': 'A', 'Y': 'C'}])

pattern4 = create_graph(['X', 'Y'], [('X', 'X'), ('Y', 'X')])
assert test_match(input, pattern4, [{'X': 'C', 'Y': 'A'}])

pattern5 = create_graph(['X', 'Y'], [('X', 'X'), ('Y', 'X'), ('X', 'Y')])
assert test_match(input, pattern5, [])

pattern6 = create_graph(['1','2','3','4','5'], [])
assert test_match(input, pattern6, [])

input.add_edge('C', 'D')
input.add_edge('D', 'A')
pattern7 = create_graph(['X', 'Y', 'Z'], [('X', 'Y'), ('Y', 'Z')])
assert test_match(input, pattern7, [{'X': 'A', 'Y': 'C', 'Z': 'D'},
                                    {'X': 'C', 'Y': 'D', 'Z': 'A'},
                                    {'X': 'D', 'Y': 'A', 'Z': 'C'},
                                    {'X': 'D', 'Y': 'A', 'Z': 'B'}])


In [ ]:
# POC: High number of nodes, solved with attribute filtering
num_nodes = 100000

input = create_graph(
    [n for n in range(num_nodes)] + [(num_nodes+1, {'attr': 15}), (num_nodes+2, {'attr': 15})], 
    [
        (num_nodes+1, num_nodes+2),
        (2,4),
        (3,1)
    ]
)

pattern = create_graph([('X', {'attr': None}), ('Y', {'attr': None})], [('X', 'Y')])
assert test_match(input, pattern, [{'X': num_nodes+1, 'Y': num_nodes+2}])